In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d emmarex/plantdisease

plantdisease.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/plantdisease.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
# Define constants
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32

# Specify target labels
target_folders = ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']


In [ ]:
import os

dataset_path = '/content/PlantVillage'

# Get the list of subdirectories (classes) in the dataset
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Print the class names
for class_name in classes:
    print(class_name)

Tomato_Leaf_Mold
Pepper__bell___healthy
Potato___Early_blight
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato_healthy
Potato___healthy
Tomato__Target_Spot
Tomato_Septoria_leaf_spot
Pepper__bell___Bacterial_spot
Tomato__Tomato_mosaic_virus
Tomato__Tomato_YellowLeaf__Curl_Virus
Tomato_Late_blight
Tomato_Early_blight
Tomato_Bacterial_spot
Potato___Late_blight


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load PlantVillage dataset using ImageDataGenerator
#train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)



In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/PlantVillage',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # since we  have multiple classes
    subset='training',
    classes=target_folders
)

validation_generator = train_datagen.flow_from_directory(
    '/content/PlantVillage',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=target_folders
)

Found 3703 images belonging to 5 classes.
Found 924 images belonging to 5 classes.


In [ ]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top=False,
    input_shape=(150,150,3)
)

In [ ]:
model = Sequential()

model.add(conv_base)

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(len(train_generator.class_indices),activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=validation_generator)

Epoch 1/5
  1/116 [..............................] - ETA: 1:09:14 - loss: 1.6137 - accuracy: 0.3438

KeyboardInterrupt: 

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

In [ ]:
# Function to preprocess an individual image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to make predictions on an individual image
def predict_image(img_path):
    processed_img = preprocess_image(img_path)
    predictions = model.predict(processed_img)
    print(target_folders[np.argmax(predictions)])
    print(np.argmax(predictions))
   # decoded_predictions = decode_predictions(predictions, top=3)[0]
    #return decoded_predictions

# Test the model on an example image
img_path = '/content/Pepper__bell___healthy.jpg'
predictions = predict_image(img_path)



In [ ]:
# Function to preprocess an individual image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to make predictions on an individual image
def predict_image(img_path):
    processed_img = preprocess_image(img_path)
    predictions = model.predict(processed_img)
    print(np.argmax(predictions))
    print(classes[np.argmax(predictions)])
   # decoded_predictions = decode_predictions(predictions, top=3)[0]
    #return decoded_predictions

# Test the model on an example image
img_path = '/content/Pepper__bell___healthy.jpg'
predictions = predict_image(img_path)



In [ ]:
# Display the predictions
for i, (imagenet_id, label, score) in enumerate(predictions):
    print(f"{i + 1}: {label} ({score:.2f})")